In [22]:
# ---------------------------------------------- Imports ----------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

In [23]:
# ---------------------------------------------- Init ----------------------------------------
try:
    client = RemoteAPIClient()
    sim = client.require('sim')
except:
    print('Error connection')

In [24]:
def readSensorData(range_data_signal_id="hokuyo_range_data", 
                    angle_data_signal_id="hokuyo_angle_data"):
    
    signalName = sim.waitForSignal(range_data_signal_id)
    
    string_range_data = sim.getStringSignal(range_data_signal_id)

    string_angle_data = sim.getStringSignal(angle_data_signal_id)

    # verifique se ambos os dados foram obtidos corretamente
    if string_range_data != None and string_angle_data != None:
        # descompacte dados de mensagens de alcance e sensor
        raw_range_data = sim.unpackFloatTable(string_range_data)
        raw_angle_data = sim.unpackFloatTable(string_angle_data)

        return raw_range_data, raw_angle_data

    # retornar nenhum caso nada tenha sido obtido do simulador
    return None

In [25]:
# #FORÇAS
# def att_force(q, goal, k=2):
#     f = k*(goal - q)
#     return f

# def rep_force(q, obs, R=2, krep=.1):
#     #q = posição do robo
#     #piq = d
#     #p0 = R é a partir de qual distancia a força de repulsao faz efeito
#     #obs = posição do obstaculo

#     Frep_total = np.array([0.0, 0.0])
#     for obstacle in obs:
#         # Calcula o vetor da posição do robô para a posição do obstáculo
#         v = q - obstacle
#         # Calcula a distância entre o robô e o obstáculo
#         d = np.linalg.norm(v)
#         print("aqui", d)
#         # Calcula a força de repulsão apenas se a distância for menor que R
#         if (d < R):  
#             rep = (1/d**2)*((1/d)-(1/R))*(v/d) 
#             Frep_total += rep

#     return krep*Frep_total

In [26]:
# #CONTROLADOR

# #PEGAMOS AS INFO DO NOVO ROBO
# def controlador(Ft_x, Ft_y):
#     robotname = 'Pioneer_p3dx'
#     robotHandle = sim.getObject("/" + robotname)  
#     robotLeftMotorHandle = sim.getObject("/" +robotname + '_leftMotor')
#     robotRightMotorHandle = sim.getObject("/" + robotname + '_rightMotor')

#     # robotPos = sim.getObjectPosition(robotHandle)
#     # print("aqui", robotPos)
#     # robotOri = sim.getObjectOrientation(robotHandle)

#     # Goal position (x, y)
#     #AQUI EU PASSO SO A POSIÇÃO QUE O GOAL TA, NAO PRECISA DA ORIENTAÇÃO,
#     #O ROBO PODE CHEGAR COM QUALQUER ORIENTAÇÃO
#     # pgoal = np.array([1, 0])
#     # pgoal = np.array([0, 0])

#     # # Frame que representa o Goal
#     # print("aqui", pgoal)
#     # goalFrame = sim.getObjectPosition(pgoal)
#     # print("ii", goalFrame)  
#     # returnCode = sim.setObjectPosition(goalFrame, [pgoal[0], pgoal[1], 0])
#     # returnCode = sim.setObjectOrientation(goalFrame, [0, 0, 0])    

#     # Específico do robô
#     # https://www.generationrobots.com/media/Pioneer3DX-P3DX-RevA.pdf
#     L = 0.381
#     r = 0.0975
#     maxv = 1.0 #LIMITADORES DA VELOCIDADE LINEAR
#     maxw = np.deg2rad(45) #LIMITADORES DA VELOCIDADE ANGULAR

#     rho = np.inf
#     while rho > .05:
#         robotPos = sim.getObjectPosition(robotHandle) #pego a posição em relação ao mundo
#         robotOri = sim.getObjectOrientation(robotHandle)        
#         robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]])     
#         dist = x_goal - robotPos
#         dist = np.linalg.norm(dist) 
#         Fatt = att_force(robotPos, x_goal)
#         Frep = rep_force(robotPos, obst)
#         Ft = Fatt + Frep
#         Ft_x = Ft[0]
#         Ft_y = Ft[1]
   
#         # dx, dy = pgoal - robotConfig[:2] #aqui eu vou pegar o erro, ou seja, a distancia entre o robo e o goal

#         # Apenas para interromper o loop
#         rho = np.sqrt(Ft_x**2 + Ft_y**2)
#         kr = 1
#         kt = 2
        
#         v = kr*(Ft_x*np.cos(robotConfig[2]) + Ft_y*np.sin(robotConfig[2]))
#         w = kt*(np.arctan2(Ft_y,Ft_x) - robotConfig[2])
                
#         # Limit v,w to +/- max
#         v = max(min(v, maxv), -maxv)
#         w = max(min(w, maxw), -maxw)        
        
#         vr = ((2.0*v) + (w*L))/(2.0*r)
#         vl = ((2.0*v) - (w*L))/(2.0*r)
#         print("oi")
#         sim.setJointTargetVelocity(robotRightMotorHandle, vr)
#         sim.setJointTargetVelocity(robotLeftMotorHandle, vl)

#     sim.setJointTargetVelocity(robotRightMotorHandle, 0)
#     sim.setJointTargetVelocity(robotLeftMotorHandle, 0)
#     print("parou")

In [27]:
#Calcula a força de atração 
def att_force(q, goal, k=2):
    f = k*(goal - q)
    return f

#Calcula a força de repulsão
def rep_force(q, obs, R=3, krep=.1):
    Frep_total = np.array([0.0, 0.0])
    for obstacle in obs:
        v = q - obstacle
        d = np.linalg.norm(v) # Calcula a distância entre o robô e o obstáculo

        # Calcula a força de repulsão apenas se a distância for menor que R
        if (d < R):  
            rep = (1/d**2)*((1/d)-(1/R))*(v/d) 
            Frep_total += rep

    return krep*Frep_total

In [28]:
#CONTROLADOR
import time

sim.startSimulation()
robotname = 'Pioneer_p3dx'
robotHandle = sim.getObject("/" + robotname)  
robotLeftMotorHandle = sim.getObject("/" +robotname + '_leftMotor')
robotRightMotorHandle = sim.getObject("/" + robotname + '_rightMotor')

x_goal = np.array([-2.41947, -2.41947])
L = 0.381
r = 0.0975
maxv = 1.0 #LIMITADORES DA VELOCIDADE LINEAR
maxw = np.deg2rad(45) #LIMITADORES DA VELOCIDADE ANGULAR

rho = np.inf
random_force = 0.1
while rho > .05:
    raw_range_data, raw_angle_data = readSensorData()
    robotPos = sim.getObjectPosition(robotHandle, sim.handle_world)
    robotPos = robotPos[0:2]
    robotOri = sim.getObjectOrientation(robotHandle, sim.handle_world)        
    robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]])

    #Obter as coordenadas do obstáculo em relação ao mundo
    obstacle_x = raw_range_data * np.cos(raw_angle_data)
    obstacle_y = raw_range_data * np.sin(raw_angle_data)
    obstacle_world_x = obstacle_x * np.cos(robotOri[2]) - obstacle_y * np.sin(robotOri[2])
    obstacle_world_y = obstacle_x * np.sin(robotOri[2]) + obstacle_y * np.cos(robotOri[2])

    # Adicione a posição do robô em relação ao mundo às coordenadas do obstáculo para obter as coordenadas do obstáculo em relação ao mundo
    obstacle_world_x += robotPos[0]
    obstacle_world_y += robotPos[1]
    obstacle = np.array([obstacle_world_x, obstacle_world_y]).T
    print("OPA", obstacle)
    
    Fatt = att_force(robotPos, x_goal)
    Frep = rep_force(robotPos, obstacle)

    random_force_vector = random_force * np.random.rand(2)
    Ft = Fatt + Frep + random_force_vector
    Ft_x = Ft[0]
    Ft_y = Ft[1]

    # Apenas para interromper o loop
    rho = np.sqrt(Ft_x**2 + Ft_y**2)
    
    # Formula De Luca e Oriolo para calcular a velocidade linear e angular.
    kr = 1
    kt = 2
    v = kr*(Ft_x*np.cos(robotConfig[2]) + Ft_y*np.sin(robotConfig[2]))
    w = kt*(np.arctan2(Ft_y,Ft_x) - robotConfig[2])
            
    # Limit v,w to +/- max
    v = max(min(v, maxv), -maxv)
    w = max(min(w, maxw), -maxw)        
    
    vr = ((1.0*v) + (w*L))/(2.0*r)
    vl = ((1.0*v) - (w*L))/(2.0*r)
    sim.setJointTargetVelocity(robotRightMotorHandle, vr)
    sim.setJointTargetVelocity(robotLeftMotorHandle, vl)

sim.setJointTargetVelocity(robotRightMotorHandle, 0)
sim.setJointTargetVelocity(robotLeftMotorHandle, 0)

time.sleep(2)
sim.stopSimulation()

KeyboardInterrupt: 

In [ ]:
# # navegação
# def navigation(q, x_goal, obst):
#     dist = x_goal - q
#     dist = np.linalg.norm(dist) 
#     dist_min = 0.1
#     while dist > dist_min:
#         print("posição", q)
#         Fatt = att_force(q, x_goal)
#         Frep = rep_force(q, obst)
#         Ft = Fatt + Frep
#         Ft_x = Ft[0]
#         Ft_y = Ft[1]
#         controlador(Ft_x, Ft_y)

In [ ]:
# sim.startSimulation()
# #INFORMAÇÕES DO ROBO
# robotname = 'Pioneer_p3dx'
# robotHandle = sim.getObject("/" + robotname)  
# robotLeftMotorHandle = sim.getObject("/" +robotname + '_leftMotor')
# robotRightMotorHandle = sim.getObject("/" + robotname + '_rightMotor')

# robotPos = sim.getObjectPosition(robotHandle) #PEGO A POSIÇÃO DO ROBO NO MUNDO PRA TACAR NO TREM DE REPULSAO
# # print("aqui", robotPos[0:2])

# raw_range_data, raw_angle_data = readSensorData()
# laser_data = np.array([raw_angle_data, raw_range_data]).T

# obstacle_x = raw_range_data * np.cos(raw_angle_data)
# obstacle_y = raw_range_data * np.sin(raw_angle_data)
# obstacle = np.array([obstacle_x, obstacle_y]).T

# x_goal = np.array([1, 0])
# Frep = navigation(robotPos[0:2], x_goal, obstacle)
# print(Frep)
# sim.stopSimulation()